In [1]:
import io
import csv
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from object_detection.utils import dataset_util

In [2]:
COLOR_RED = 'red'
COLOR_GREEN = 'green'
COLOR_YELLOW = 'yellow'

COLOR_RED_NUM = 1
COLOR_GREEN_NUM = 2
COLOR_YELLOW_NUM = 3

def getLabel(init_label):
    return {
        'stop': COLOR_RED,
        'go': COLOR_GREEN,
        'goLeft': COLOR_GREEN,
        'warning': COLOR_YELLOW,
        'warningLeft': COLOR_YELLOW,
        'stopLeft': COLOR_RED,
        'goForward': COLOR_GREEN
    }[init_label]

def getNumericLabel(text_label):
    return {
        COLOR_RED: COLOR_RED_NUM,
        COLOR_GREEN: COLOR_GREEN_NUM,
        COLOR_YELLOW: COLOR_YELLOW_NUM
    }[text_label]

In [3]:
def getAnnotations(images_path, annotations_path):
    global filenames, labels, boxes
    with open(annotations_path) as annotations:
        reader = csv.DictReader(annotations, delimiter=';')
        for row in reader:
            filenames.append(images_path + (row['Filename'].split('/'))[1])
            labels.append(getLabel(row['Annotation tag']))
            boxes_tmp = [row['Upper left corner X'], row['Upper left corner Y'], row['Lower right corner X'], row['Lower right corner Y']]
            boxes.append(boxes_tmp)

In [4]:
labels_path = './data/Annotations/'

filenames = []
labels = []
boxes = []
images = []

images_path = './data/daySequence1/frames/'
annotations_path = labels_path + 'daySequence1/frameAnnotationsBOX.csv'
getAnnotations(images_path, annotations_path)
        
images_path = './data/daySequence2/frames/'
annotations_path = labels_path + 'daySequence2/frameAnnotationsBOX.csv'
getAnnotations(images_path, annotations_path)
print(filenames[0])
print('Number of files: ' + str(len(filenames)))

./data/daySequence1/frames/daySequence1--00000.jpg
Number of files: 18020


In [19]:
def generateImages():
    i=-1
    for file in filenames:
        i+=1
        image = Image.open(file)
        with tf.gfile.GFile(file, 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        w, h = image.size[:2]
        if image is None:
            print('error')
        else:
            yield i, file, encoded_jpg, w, h

In [20]:
def prepareTFRecord(height, width, filename, image, image_format, boxes, labels_text, labels):
    xmin, ymin, xmax, ymax = list(map(int, boxes))
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature([xmin/w]),
        'image/object/bbox/xmax': dataset_util.float_list_feature([xmax/w]),
        'image/object/bbox/ymin': dataset_util.float_list_feature([ymin/h]),
        'image/object/bbox/ymax': dataset_util.float_list_feature([ymax/h]),
        'image/object/class/text': dataset_util.bytes_list_feature([labels_text]),
        'image/object/class/label': dataset_util.int64_list_feature([labels]),
    }))
    return tf_example

In [22]:
#get TFRecords:
writer = tf.python_io.TFRecordWriter('train.record')
for i, f, img, w ,h in generateImages():
    raw_filename = f.encode('utf8')
    raw_label = labels[i].encode('utf8')
    raw_label_num = getNumericLabel(labels[i])
    TFRecord = prepareTFRecord(h, w, raw_filename, img, b'jpg', boxes[i], raw_label, raw_label_num)
    writer.write(TFRecord.SerializeToString())
print("{} records saved.".format(i))
writer.close()
print('TFRecords saved')

18019 records saved.
TFRecords saved


In [8]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('boxes', 158200),
 ('filenames', 158200),
 ('labels', 158200),
 ('img', 136715),
 ('generateImages', 136),
 ('getAnnotations', 136),
 ('getLabel', 136),
 ('getNumericLabel', 136),
 ('prepareTFRecord', 136),
 ('annotations_path', 104),
 ('f', 99),
 ('TFRecord', 88),
 ('raw_filename', 83),
 ('Image', 80),
 ('dataset_util', 80),
 ('np', 80),
 ('tf', 80),
 ('images_path', 76),
 ('labels_path', 68),
 ('images', 64),
 ('writer', 56),
 ('COLOR_YELLOW', 55),
 ('COLOR_GREEN', 54),
 ('COLOR_RED', 52),
 ('raw_label', 38),
 ('COLOR_GREEN_NUM', 28),
 ('COLOR_RED_NUM', 28),
 ('COLOR_YELLOW_NUM', 28),
 ('h', 28),
 ('i', 28),
 ('raw_label_num', 28),
 ('w', 28)]

In [9]:
print(set(labels))

{'yellow', 'green', 'red'}
